1. 学習
   1. データの前処理
   2. 誤差を求める（損失関数）
   3. 誤差を減らすために勾配の算出
   4. パラメータの更新
2. 推論

# 畳み込みニューラルネットワーク(convolutional neural network : CNN)

画像認識や音声認識で使われている

## これまで扱ってきた全結合のネットワーク

Affineレイヤを計算グラフで表す  

```mermaid
graph LR
    A[X] --> B[dot]
    B -->|X・W| C[＋]
    C --> D[Y]
    E[W] --> B
    F[B] --> C

    D -->|dL/dy = 1| C
    C --> B
    B --> A
```
MNISTデータセットを使ったときの、X dot W　を細かく見ると

<img width="50%" src="隣接するニューロン.png">

このように、これまでは、隣接するすべてのニューロン間で結合があった  

全結合によるネットワーク像（例）
```mermaid
graph LR
    A[2] ---> B[Affine]
    B --> C[ReLU]
    C ---> D[Affine]
    D --> E[ReLU]
    E ---> F[Affine]
    F --> G[ReLU]
    G ---> H[Affine]
    H --> I[Softmax]
```
Softmaxレイヤで最終的な結果を出力する。


***

### 全結合の問題点


1. データの形状が無視されてしまう  
MNISTのデータセットを使った際は、$28 \times 28$ ピクセルの画像を、784個データにしてAffineに入力した  
→　１次元のデータになおしてしまうので、２次元・３次元空間でのピクセルの近さなどの情報が含まれずに学習されてしまう。  

例えばこのような猫をニューラルネットワークによって画像認識できるように学習したいとする。

<img width="50%" src="cat.png">

これを説明上わかりやすくするために、雑にピクセル化します。

<img width="50%" src="cut_cat.png">

これを１次元で学習するということは、下のような１列のデータを学習するということと近いと思います。

<img width="50%" src="1ndim_cat.png">

このように一列だと、猫の耳のピクセル値が行列のどこになるかは、画像全体に対する猫の位置やサイズにもよりそうです。  
また、猫の顔全体に対するそれぞれのパーツの位置関係や形などの猫ならではの特徴が効果的に学習できなさそうなのは想像に易いと思います。

話を全結合層に戻し、正確な表現をすると、  
全結合層だと、形状を無視して、すべての入力データを同等のニューロン（同じ次元のニューロン）として扱うので、  
下記のような３次元ならではの形状に関する情報を活かすことができません。

- RGBの各チャンネル間の密接な関連性
- ピクセルの距離


<img width="50%" src="angry_cat.png">


## CNNの全体の構造

CNNは次の２つの層で構成されます。

$$
「Convolutionレイヤ（畳み込み層）」　＋　「Poolingレイヤ（プーリング層）」
$$

***

#### CNNによるネットワークの例
```mermaid
graph LR
    A[2] ---> B[Conv]
    B --> C[ReLU]
    C --> D[Pooling]
    D ---> E[Conv]
    E --> F[ReLU]
    F --> G[Pooling]
    G ---> H[Conv]
    H --> I[ReLU]
    I ---> J[Affine]
    J --> K[ReLU]
    K ---> L[Affine]
    L --> M[Softmax]
```

## 畳み込み層

畳み込み層では、入力データを３次元で受け取り、３次元のデータとして出力できる。  
そのため、CNNでは画像のような形状を持ったデータを正しく学習できる可能性があります。  

■用語の説明  
・畳み込み層の入出力データ：特徴マップ  
・畳み込み層の入力データ：入力特徴マップ  
・畳み込み層の出力データ：出力特徴マップ  

### 畳み込み演算


実際に畳み込み層で行われている畳み込み演算手順について説明します。

#### 1. 入力データとフィルター（カーネル）の用意（縦横２次元の入力データとフィルター）
<img width="50%" src="入力データとフィルター.png">



#### 2. 一定の間隔でフィルターのウィンドウをスライドさせて畳み込み演算を行う
<img width="50%" src="フィルタースライド.png">

画像ではスライドの間隔を１にしてスライドしています。  
この幅のことを**ストライド**といいます。

演算方法  
フィルターの要素と、対応する入力の要素を乗算し、その和を求める(積和演算)  

<img width="50%" src="畳み込み演算.png">

#### 3. バイアス

- バイアスはつねにひとつ（1✕1）
- すべての要素に加算される

<img width="50%" src="バイアス.png">

基本的な畳み込み演算の手順は以上になります。

ただ、画像の通り(4, 4)のサイズの入力データが、畳み込み演算により、(2, 2)となっており、  
データが縮小していることがわかります。  
畳み込み演算を何度も繰り返すネットワークだと、いずれ出力サイズが1になってしまいそうです。  

それを回避するための手法が次で紹介するパディングになります。

### パディング



入力データの周囲に固定のデータ(例えば0)を埋めること  

<img width="50%" src="パディング.png">

画像のように、畳み込み演算をしても、サイズを一定にしたまま次の層へデータを渡すことができる。

### 畳み込み演算設定値の関係性を式で表してみる



$$
出力縦 = \frac{入力縦幅 + 2 \times パディング - フィルター縦幅}{ストライド} + 1
$$

$$
出力横 = \frac{入力横幅 + 2 \times パディング - フィルター横幅}{ストライド} + 1
$$

※注意  
それぞれ割り切れるように値を設定する必要がある。  
割り切れないときは最も近い整数に丸めるなどしてエラーを出さずに先に進める実装もある

### 3次元データの畳み込み演算

実際の画像データは、縦・横方向に加えて、色味というチャンネル方向も合わせた3次元データを扱う必要があります。

<img width="50%" src="３次元畳み込み.png">

チャンネルごとに入力データとフィルターの畳み込み演算を行い、それらの結果を加算して一つの出力を得ます。  
あとは、２次元のときと同じようにフィルターをスライドさせて同じ計算を繰り返します。


※注意点  
- 入力データとフィルターのチャンネル数は同じにする
- フィルターのサイズは自由
- チャンネルごとのフィルターのサイズはすべて同じ
  - （Rチャンネルのフィルターサイズが(3,3)なら他のRチャンネルも同じ）


この例では、３つのチャンネルを持つ入力に対して、チャンネル数が１の特徴マップが出力されます。  
しかし、これではパディングのときと同様に出力の要素が縮小しています。  
出力をチャンネル方向に複数持たせるために、複数のフィルターをもたせます。

#### 複数のフィルターによる畳み込み

以下で３つのフィルターを持たせています。

<img width="50%" src="複数のフィルター.png">

こうすれば、出力をチャンネル方向に複数持たせることができます。

バイアス項も追加すると、処理フローは以下のようになります。

<img width="50%" src="３次元畳み込みバイアス.png">

#### バッチ処理にすると

<img width="50%" src="3次元バッチ畳み込み.png">

バッチ処理にすることで、複数のデータを一度の処理でまとめて計算できる。  
これは本冒頭で説明があった、Numpyの行列計算機能のおかげです。  
この行列計算機能は、ループを使わずに、配列全体に対して一度に操作を行うことができます。

### 所感


- 理解できた点
  - 畳み込み演算の方法はわかった
  - 畳み込み演算によりデータの形状をそのままに学習できることがわかった
  - 研究で使っていたGCN類の手法の具体的な計算方法が理解できた

## プーリング層

縦・横方向のサイズ・空間を小さくする演算

### プーリング層の特徴

- 学習するパラメータがない
  - 対象から最大値（もしくは平均値）を取るだけ
- チャンネル数は変化しない
- 微小な変化に対して頑健（ロバスト）

### maxプーリング
- ノイズの影響を受けにくい
- 際立った情報を捉えることができる
- 情報損失量が多い

### averageプーリング


発表者所感  
層を通すごとに情報が抽象的になる。高度になっていくイメージ。  
